<a href="https://colab.research.google.com/github/22ananya/MUSI6201/blob/main/Final%20Project%20-%20Stereo%20Demixing/generate_and_save_reference_tracks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import generally required packages - update as needed

In [1]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as sp
import scipy.io.wavfile as wav

Import/Install the prerequisite code for implementing the Cadenza challenge - includes baselines, other important files

In [2]:
!pip install pyclarity==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 13.8 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filenam

Import the dataset for the Cadenza challenge directly through the Google Drive link - Only needs to be done once! So now commented out


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check current path

In [4]:
import os
print(os.getcwd())

/content


Change current path to Cadenza folder

In [5]:
os.chdir('/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024')
print(os.getcwd())

/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024


#Process single audio file through the entire model step by step

Import all dependencies - same as enhance.py file

In [6]:
from __future__ import annotations

import json
import logging
from pathlib import Path

# pylint: disable=import-error
import hydra
import numpy as np
import torch
from numpy import ndarray
from omegaconf import DictConfig
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB

from clarity.enhancer.compressor import Compressor
from clarity.enhancer.nalr import NALR
from clarity.evaluator.haaqi import compute_haaqi
from clarity.utils.audiogram import Audiogram, Listener
from clarity.utils.file_io import read_signal
from clarity.utils.flac_encoder import FlacEncoder
from clarity.utils.results_support import ResultsFile
from clarity.utils.signal_processing import (
    clip_signal,
    denormalize_signals,
    normalize_signal,
    resample,
    to_16bit,
    compute_rms,
    resample,
)
from clarity.utils.source_separation_support import get_device, separate_sources
from recipes.cad_icassp_2024.baseline.evaluate import (
    apply_gains,
    apply_ha,
    make_scene_listener_list,
    remix_stems,
    load_reference_stems,
)

logger = logging.getLogger(__name__)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/evaluate.py:190: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import all the required functions defined in Enhance.py that do not need to be changed

In [7]:
from recipes.cad_icassp_2024.baseline.enhance import (
    save_flac_signal,
    decompose_signal,
    process_remix_for_listener
)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/enhance.py:182: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import the correct config file (hardcoded location)

In [8]:
from omegaconf import OmegaConf
config = OmegaConf.load('baseline/config.yaml')
print(config.separator.model)

demucs


Set output directory

In [9]:
enhanced_folder = Path("NEW_Ref_Tracks")
enhanced_folder.mkdir(parents=True, exist_ok=True)

Load listener data

In [10]:
config.path.root = '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024'
print(config.path)


{'root': '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024', 'metadata_dir': '${path.root}/metadata', 'music_dir': '${path.root}/audio/at_mic_music', 'gains_file': '${path.metadata_dir}/gains.json', 'head_positions_file': '${path.metadata_dir}/head_positions.json', 'listeners_file': '${path.metadata_dir}/listeners.train.json', 'music_file': '${path.metadata_dir}/at_mic_music.train.json', 'scenes_file': '${path.metadata_dir}/scenes.train.json', 'scene_listeners_file': '${path.metadata_dir}/scene_listeners.train.json', 'exp_folder': './exp'}


In [11]:
# Load listener audiograms and songs
listener_dict = Listener.load_listener_dict(config.path.listeners_file)

with Path(config.path.gains_file).open("r", encoding="utf-8") as file:
    gains = json.load(file)

with Path(config.path.scenes_file).open("r", encoding="utf-8") as file:
    scenes = json.load(file)

with Path(config.path.scene_listeners_file).open("r", encoding="utf-8") as file:
    scenes_listeners = json.load(file)

with Path(config.path.music_file).open("r", encoding="utf-8") as file:
    songs = json.load(file)

Load the enhancer (NAL-R) and compression setting (OFF)

In [12]:
enhancer = NALR(**config.nalr)
compressor = Compressor(**config.compressor)

Create a list of songs, listeners (audiogram) and head position (hrtf) to generate, or evaluate the mix - based on provided data

In [13]:
# Select a batch to process
scene_listener_pairs = make_scene_listener_list(
    scenes_listeners, config.evaluate.small_test
)

scene_listener_pairs = scene_listener_pairs[
    config.evaluate.batch :: config.evaluate.batch_size
]

In [14]:
print(scene_listener_pairs[0])

('scene_10001', 'L0066')


# Load functions required for audio evaluation

# Process Audio to generate the reference files - Currently set to process a fixed number of runs (listener - scene pairings, can be changed to run entire dataset)

In [15]:
from recipes.cad_icassp_2024.baseline.evaluate import level_normalisation
previous_song = ""
num_scenes = len(scene_listener_pairs)
nrun = 20 # change to num_scenes for full dataset

for idx in range(nrun):
    scene_id, listener_id = scene_listener_pairs[idx]

    scene = scenes[scene_id]
    song_name = f"{scene['music']}-{scene['head_loudspeaker_positions']}"

    logger.info(
        f"[{idx:03d}/{num_scenes:03d}] "
        f"Processing {scene_id}: {song_name} for listener {listener_id}"
    )
    # Get the listener's audiogram
    listener = listener_dict[listener_id]


    # Load reference file and process same as demixed stems
    # Load reference signals - make sure this points to the correct path of reference hrtf applied music
    reference_stems, original_mixture = load_reference_stems(
        Path(config.path.music_dir) / songs[song_name]["Path"]
    )
    reference_stems = apply_gains(
        reference_stems, config.sample_rate, gains[scene["gain"]]
    )
    reference_mixture = remix_stems(
        reference_stems, original_mixture, config.sample_rate
    )

    # Apply hearing aid to reference signals
    left_reference = apply_ha(
        enhancer=enhancer,
        compressor=None,
        signal=reference_mixture[:, 0],
        audiogram=listener.audiogram_left,
        apply_compressor=False,
    )
    right_reference = apply_ha(
        enhancer=enhancer,
        compressor=None,
        signal=reference_mixture[:, 1],
        audiogram=listener.audiogram_right,
        apply_compressor=False,
    )

    final_mixture = np.stack([left_reference, right_reference], axis=1)
    final_mixture = level_normalisation(final_mixture, original_mixture, config.sample_rate)


    # Write to file! - Make sure the enhanced_folder directory is correct

    filename = Path(enhanced_folder) / f"{scene_id}_{listener.id}_remix_ref.flac"

    filename.parent.mkdir(parents=True, exist_ok=True)
    save_flac_signal(
        signal=final_mixture,
        filename=filename,
        signal_sample_rate=config.sample_rate,
        output_sample_rate=config.remix_sample_rate,
        do_clip_signal=True,
        do_soft_clip=config.soft_clip,
        )


logger.info("Done!")